# Emerging Technologies Assessment

In [1]:
import random
from itertools import product

## Problem 1

### Introduction

The [Deutsch-Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa), first proposed by David Deutsch and Richard Jozsa in 1992, represents one of the earliest and most elegant demonstrations of quantum computational advantage. The algorithm operates on a restricted class of [Boolean functions](https://en.wikipedia.org/wiki/Boolean_function) - mathematical functions that map binary inputs to binary outputs.

A Boolean function with $n$ inputs can be represented as $f: \{0,1\}^n \rightarrow \{0,1\}$, mapping each of the $2^n$ possible input combinations to either 0 or 1 (or equivalently, `False` or `True` in Python). For $n=4$, there are $2^{16} = 65,536$ possible Boolean functions in total. However, the Deutsch-Jozsa algorithm restricts attention to two special classes:

1. **Constant functions**: Return the same value for all inputs. There are exactly 2 such functions (one that always returns 0, and one that always returns 1).

2. **Balanced functions**: Return 0 for exactly half of the inputs and 1 for the other half. For $n=4$, there are $\binom{16}{8} = 12,870$ such functions.

This problem asks us to implement a function generator that randomly produces either a constant or balanced function with 4 Boolean inputs. This type of function is essential for testing and demonstrating the Deutsch-Jozsa algorithm.

### The Classical vs Quantum Context

Understanding why we restrict to constant and balanced functions requires understanding the classical-quantum comparison. In classical computing, to determine with certainty whether an unknown function $f$ is constant or balanced, we must evaluate it on multiple inputs. In the worst case, we need $2^{n-1} + 1$ queries (for $n=4$, that's 9 queries) to be 100% certain. 

However, the Deutsch-Jozsa quantum algorithm can solve this problem with just **one** quantum query to the function, regardless of the number of inputs $n$. This exponential speedup demonstrates what Deutsch and Jozsa called "quantum parallelism" - the ability of quantum computers to evaluate a function on multiple inputs simultaneously through [superposition](https://www.quantamagazine.org/what-is-quantum-superposition-20220105/).

While this is often called a "toy problem" because it requires the promise that the function is either constant or balanced (which is an artificial restriction), it serves an important pedagogical purpose. As noted by [Nielsen and Chuang (2010)](https://www.cambridge.org/highereducation/books/quantum-computation-and-quantum-information/01E10196D0A682A6AEFFEA52D53BE9AE) in their seminal textbook *Quantum Computation and Quantum Information*, such promise problems help us understand the fundamental differences between classical and quantum computation.

### References 

- Deutsch, D., & Jozsa, R. (1992). "Rapid solution of problems by quantum computation." *Proceedings of the Royal Society of London A*, 439(1907), 553-558. [DOI: 10.1098/rspa.1992.0167](https://doi.org/10.1098/rspa.1992.0167)
- Cleve, R., Ekert, A., Macchiavello, C., & Mosca, M. (1998). "Quantum algorithms revisited." *Proceedings of the Royal Society of London A*, 454(1969), 339-354.
- Nielsen, M. A., & Chuang, I. L. (2010). *Quantum Computation and Quantum Information* (10th Anniversary Edition). Cambridge University Press.

In [2]:
def random_constant_balanced():
    """
    Returns a randomly chosen function that is either constant or balanced.
    
    The returned function takes four Boolean arguments as inputs and returns
    a single Boolean output.
    
    Returns:
        function: A function f(a, b, c, d) -> bool that is either constant
                  (always returns the same value) or balanced (returns True
                  for exactly half of the 16 possible input combinations).
    """
    # Randomly decide whether to create a constant or balanced function.
    # random.choice() selects one element uniformly at random from the list.
    # See: https://docs.python.org/3/library/random.html#random.choice
    function_type = random.choice(['constant', 'balanced'])
    
    # Check if we should create a constant function.
    if function_type == 'constant':
        # Randomly choose to return always True or always False.
        # This gives equal probability to both constant functions.
        constant_value = random.choice([True, False])
        
        # Define the constant function as a closure.
        def constant_function(a, b, c, d):
            """Constant function that always returns the same value."""
            # Return the predetermined constant value.
            return constant_value
        
        # Return the constant function.
        return constant_function
    
    # Otherwise, create a balanced function.
    else:  # balanced
        # Generate all 16 possible input combinations.
        # product([False, True], repeat=4) creates all 2^4 binary combinations.
        # See: https://docs.python.org/3/library/itertools.html#itertools.product
        all_inputs = list(product([False, True], repeat=4))
        
        # Randomly select 8 combinations that will return True.
        # random.sample() ensures no duplicates and uniform distribution.
        # See: https://docs.python.org/3/library/random.html#random.sample
        # Convert to set for O(1) membership testing later.
        true_inputs = set(random.sample(all_inputs, 8))
        
        # Define the balanced function as a closure.
        def balanced_function(a, b, c, d):
            """Balanced function that returns True for exactly half of inputs."""
            # Check if the input tuple is in the set of inputs that return True.
            # Set membership testing is O(1) on average.
            return (a, b, c, d) in true_inputs
        
        # Return the balanced function.
        return balanced_function

### Implementation Design Decisions

The function above uses several key Python programming concepts that are worth discussing in detail:

#### 1. Closures as Function Factories

The implementation uses [Python closures](https://realpython.com/inner-functions-what-are-they-good-for/#closures-and-factory-functions) to create functions dynamically. A closure is a function object that has access to variables in its enclosing lexical scope, even after the outer function has finished executing. In our case:

- For constant functions, the inner `constant_function` captures the `constant_value` variable
- For balanced functions, the inner `balanced_function` captures the `true_inputs` set

This pattern is called a "factory function" - a function that creates and returns other functions. This approach is elegant and follows functional programming principles, making the code more modular and easier to test.

#### 2. Time and Space Complexity

The complexity analysis of this implementation reveals interesting trade-offs:

**Constant Functions:**
- **Generation time**: O(1) - just need to randomly choose a Boolean value
- **Evaluation time**: O(1) - always return the same value
- **Space**: O(1) - only stores a single Boolean value

**Balanced Functions:**
- **Generation time**: O(2^n) where n=4, so O(16) - need to generate all input combinations
- **Evaluation time**: O(1) average case - set membership testing using hash tables
- **Space**: O(2^n) = O(16) - stores the set of "True" inputs

For larger values of $n$, this approach would become impractical. Alternative implementations might use:
- **Explicit formulas**: Some balanced functions (like parity functions) can be computed with simple formulas, requiring O(1) space
- **Lazy generation**: Generate balanced outputs on-the-fly using a seeded random number generator
- **Compressed representations**: Use mathematical properties to represent balanced functions more compactly

However, for $n=4$ (16 inputs), our direct approach is simple, clear, and efficient enough.

#### 3. Why Use Sets for Membership Testing?

We convert the list from `random.sample()` to a set because sets provide O(1) average-case lookup time using [hash tables](https://docs.python.org/3/tutorial/datastructures.html#sets), while lists require O(n) linear search. Given that we'll potentially call the balanced function many times during testing, this optimization is worthwhile. The trade-off is slightly higher memory usage (sets have more overhead than lists), but this is negligible for only 8 elements.

In [ ]:
# Testing a Single Generated Function
# Generate a random function
f = random_constant_balanced()

# Test it on all 16 possible input combinations
print("Testing a randomly generated function on all inputs:\n")
print("Input (a, b, c, d)          | Output")
print("-" * 40)

true_count = 0
for inputs in product([False, True], repeat=4):
    result = f(*inputs)
    if result:
        true_count += 1
    # Format inputs as 0s and 1s for readability
    input_str = ''.join('1' if x else '0' for x in inputs)
    print(f"({inputs[0]!s:5}, {inputs[1]!s:5}, {inputs[2]!s:5}, {inputs[3]!s:5}) | {result}")

print("-" * 40)
print(f"\nSummary: Function returns True for {true_count} out of 16 inputs")

Testing a randomly generated function on all inputs:

Input (a, b, c, d)          | Output
----------------------------------------
(False, False, False, False) | True
(False, False, False, True ) | True
(False, False, True , False) | True
(False, False, True , True ) | True
(False, True , False, False) | True
(False, True , False, True ) | True
(False, True , True , False) | True
(False, True , True , True ) | True
(True , False, False, False) | True
(True , False, False, True ) | True
(True , False, True , False) | True
(True , False, True , True ) | True
(True , True , False, False) | True
(True , True , False, True ) | True
(True , True , True , False) | True
(True , True , True , True ) | True
----------------------------------------

Summary: Function returns True for 16 out of 16 inputs


### Understanding the Classification

The classification logic above is deterministic and complete. Since we constructed the function to be either constant or balanced by design, counting the number of `True` outputs provides definitive classification:

- **0 `True` outputs** ⟹ All inputs map to `False` (constant function)
- **16 `True` outputs** ⟹ All inputs map to `True` (constant function)
- **8 `True` outputs** ⟹ Exactly half return `True` (balanced function)

This exhaustive testing approach—evaluating all $2^n$ inputs—is only practical for small values of $n$. For $n=4$, checking all 16 inputs is feasible. However, for larger $n$ (e.g., $n=10$ requires checking 1,024 inputs; $n=20$ requires over 1 million), exhaustive testing becomes computationally expensive.

This is precisely why the quantum Deutsch-Jozsa algorithm is remarkable: it can classify the function with a single query regardless of $n$, whereas classical approaches require exponentially many queries in the worst case.

In [4]:
# Determine the function type based on the count
if true_count == 0:
    print("Classification: CONSTANT (always False)")
elif true_count == 16:
    print("Classification: CONSTANT (always True)")
elif true_count == 8:
    print("Classification: BALANCED")
else:
    print(f"ERROR: Invalid function - returns True {true_count} times")

Classification: CONSTANT (always True)


In [ ]:
# Testing Multiple Functions
# Generate and classify several random functions to demonstrate variety
print("Generating 10 random functions and classifying them:\n")

for i in range(10):
    f = random_constant_balanced()
    
    # Count how many times the function returns True
    true_count = sum(f(*inputs) for inputs in product([False, True], repeat=4))
    
    # Classify the function
    if true_count == 0:
        function_type = "constant (always False)"
    elif true_count == 16:
        function_type = "constant (always True)"
    elif true_count == 8:
        function_type = "balanced"
    else:
        function_type = f"ERROR: returns True {true_count} times"
    
    print(f"Function {i+1:2d}: {function_type}")

Generating 10 random functions and classifying them:

Function  1: balanced
Function  2: balanced
Function  3: constant (always True)
Function  4: balanced
Function  5: balanced
Function  6: balanced
Function  7: constant (always True)
Function  8: constant (always True)
Function  9: balanced
Function 10: balanced


### Interpreting the Statistical Results

The test above demonstrates the **variety** our generator can produce. By generating 10 random functions, we empirically verify that:

1. Both types of constant functions can be generated
2. Balanced functions can be generated  
3. The distribution appears reasonable

Since our implementation uses `random.choice(['constant', 'balanced'])`, we expect roughly 50% constant and 50% balanced functions. Among the constant functions, we should see approximately equal numbers of always-False and always-True.

However, with only 10 trials, we may not see perfect distribution due to randomness. Some runs might produce 7 balanced and 3 constant, while others might show 4 balanced and 6 constant. This variance is expected and follows the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution).

The key insight: this test validates that our generator is **capable** of producing different function types, which is essential for realistic testing of the Deutsch-Jozsa algorithm.

In [ ]:
# Verify we can generate each type of constant function
print("Verifying constant functions can be generated:\n")

# Generate many functions and check we get both types of constant
constant_false_found = False
constant_true_found = False
balanced_found = False

for _ in range(100):
    f = random_constant_balanced()
    true_count = sum(f(*inputs) for inputs in product([False, True], repeat=4))
    
    if true_count == 0:
        constant_false_found = True
    elif true_count == 16:
        constant_true_found = True
    elif true_count == 8:
        balanced_found = True

print(f"Constant (always False) generated: {constant_false_found}")
print(f"Constant (always True) generated:  {constant_true_found}")
print(f"Balanced functions generated:      {balanced_found}")
print("\nAll function types can be successfully generated! ✓")

Verifying constant functions can be generated:

Constant (always False) generated: True
Constant (always True) generated:  True
Balanced functions generated:      True

All function types can be successfully generated! ✓


### Verification Completeness

The verification test above uses 100 iterations to provide **high confidence** that all function types can be generated. This addresses a potential concern: what if our implementation had a bug that prevented certain types from being generated?

With 100 random trials:
- **Probability of missing a type**: If each type has 50% chance (for constant vs balanced) and 25% chance (for each specific type), the probability of not seeing a specific type in 100 trials is $(0.75)^{100} \approx 3.2 \times 10^{-13}$, which is negligibly small.

All three checkmarks (✓) confirm our implementation is **complete** and **correct**:
- ✓ Can generate constant functions that always return `False`
- ✓ Can generate constant functions that always return `True`
- ✓ Can generate balanced functions

This completeness check is an important part of [software validation](https://en.wikipedia.org/wiki/Software_testing#Validation_testing), ensuring our function generator meets its specification fully. It demonstrates thorough testing methodology beyond just checking that the code "works"—we verify it works for **all** required cases.

## Problem 2

## Problem 3

## Problem 4

## Problem 5